In [1]:
"""
@author: albertigno

"""

from MyLargeDataset import *
from snn_models import *

import torch, time, os

import torch.nn as nn
import torch.nn.functional as F

import networkx as nx

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (device)

/Users/alberto/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


False
cpu


In [2]:
#os.environ['CUDA_VISIBLE_DEVICES'] = "3"
thresh = 0.3
decay = 0.3
batch_size = 256
learning_rate = 1e-4
time_window = 50 # shd
#time_window = 15 # shd
dataset_path = r'./../../datasets'

best_acc = 0
acc_record = list([])

In [3]:
print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (device)
train_path = dataset_path+'/shd_digits/shd_train.h5'
#train_path = dataset_path+'/shd_digits/shd_test.h5'
test_path = dataset_path+'/shd_digits/shd_test.h5'
# load datasets
print("loading test set...")
test_dataset = MyDataset(test_path, 'hd_digits', time_window, device)
print("loading training set...")
train_dataset = MyDataset(train_path, 'hd_digits', time_window, device)
print("loading data with pytorch")
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False, drop_last=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True, drop_last=True)

False
cpu
loading test set...
torch.Size([2264, 50, 700])
torch.Size([2264])
num sample: 2264
torch.Size([2264, 50, 700]) torch.Size([2264, 20])
loading training set...


KeyboardInterrupt: 

In [ ]:
snn = RSNN_delay(d='shd', num_hidden=64, thresh=0.3, decay=0.3, batch_size=batch_size, win=time_window, device=device)
#snn = RSNN(d='shd', num_hidden=64, thresh=0.3, decay=0.3, batch_size=batch_size, win=time_window, device=device)

In [ ]:
num_epochs = 50
    
snn.to(device)

criterion = nn.MSELoss()

modelname = 'shd_rnn_{}.t7'.format(snn.num_hidden)

def lr_scheduler(optimizer, epoch, init_lr=0.1, lr_decay_epoch=100):
    """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""

    if epoch % lr_decay_epoch == 0 and epoch > 1:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.98

    return optimizer

optimizer = torch.optim.Adam(snn.parameters(), lr=learning_rate)

act_fun = ActFun.apply
print(modelname)

In [ ]:
best_acc = 0
train_loss = []
test_loss = []

if not os.path.isdir('checkpoint'):
    os.mkdir('checkpoint')

    
#torch.autograd.set_detect_anomaly(True)
    
for epoch in range(num_epochs):
    running_loss = 0
    total_loss_train = 0
    start_time = time.time()
    total = 0
    
    for i, (images, labels) in enumerate(train_loader):
        
        snn.zero_grad()
        optimizer.zero_grad()

        images = images.float().to(device)
        labels = labels.float().to(device)
        outputs = snn(images)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        total_loss_train += loss.item()
        total += labels.size(0)
        loss.backward()
        optimizer.step()
        if (i + 1) % 50 == 0:
            # print( outputs.sum(dim=0) )
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.5f'
                  % (epoch + 1, num_epochs, i + 1, len(train_dataset) // batch_size, running_loss))
            running_loss = 0
            print('Time elasped:', time.time() - start_time)
    train_loss.append(total_loss_train / total)
    
    optimizer = lr_scheduler(optimizer, epoch, learning_rate, 1)
    
    correct = 0
    total = 0
    total_loss_test = 0
    for images, labels in test_loader:
        images = images.float().to(device)
        labels = labels.float().to(device)
        outputs = snn(images)
        loss = criterion(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)
        _, reference = torch.max(labels.data, 1)
        total += labels.size(0)
        correct += (predicted == reference).sum()
        total_loss_test += loss.item() 

    print('Iters:', epoch, '\n\n\n')
    print('Test Accuracy of the model on the test samples: %.3f' % (100 * correct.float() / total))
    
    print('Train loss: {}, Test loss: {}'.format(total_loss_train, total_loss_test))
    
    acc = 100. * float(correct) / float(total)
    acc_record.append(acc)
    
    test_loss.append(total_loss_test / total)
    state = {
    'net': snn.state_dict(),
    'acc': acc,
    'epoch': epoch,
    'acc_record': acc_record,
    'train_loss': train_loss,
    'test_loss': test_loss
    }    
    
    if acc>best_acc:
        print('Saving best accuracy so far..')
        print(acc)
        torch.save(state, './checkpoint/' + modelname,  _use_new_zipfile_serialization=False)
        best_acc = acc

In [ ]:
snn.d_ho.weight

In [ ]:
state = {
'net': snn.state_dict(),
'acc': acc,
    
    
    
'epoch': epoch,
'acc_record': acc_record,
'train_loss': train_loss,
'test_loss': test_loss
}    

mf = modelname.split('.')[0] + '_final.t7'
torch.save(state, './checkpoint/' + mf ,  _use_new_zipfile_serialization=False)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_loss)
plt.plot(test_loss)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_loss)
plt.plot(test_loss)

In [ ]:
def get_accuracy(model):
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.float().to(device)
        outputs = model(images, win)
        _, predicted = torch.max(outputs.data, 1)
        _, labels = torch.max(labels.data, 1)
        total += labels.size(0)
        correct += (predicted.cpu() == labels).sum()
    
    acc = 100 * correct.float() / total
    return acc

acc = get_accuracy(snn)

print('Test Accuracy of the model on the 10000 test images: %.3f' % (acc))

In [ ]:
## 

In [ ]:
import torch
import numpy as np
import os

print("loading trained model")
model_to_load = modelname

#snn_state_dict = torch.load('./checkpoint/'+modelname, map_location=torch.device('cpu'))['net']
snn_state_dict = torch.load('./checkpoint/'+model_to_load, map_location=torch.device('cpu'))['net']
print(snn_state_dict.keys())

layers_location = 'checkpoint/'+model_to_load.split('.')[0]

if not os.path.isdir(layers_location):
    os.mkdir(layers_location)

weights_biases = []
for k in snn_state_dict:
    np.savez(layers_location+'/'+k,snn_state_dict[k].data.cpu().numpy())
    weights_biases.append(snn_state_dict[k].data.cpu().numpy())

In [ ]:
model_to_load = modelname
#model_to_load = 'heidelberg_rnn_400_mem1.t7'
snn_state_dict = torch.load('./checkpoint/'+model_to_load, map_location=torch.device('cpu'))['net']

model = SNN_Model()
model.load_state_dict(snn_state_dict)
model.to(device)
model.eval()

# run a subset of test set
num_to_test = 50
for images, labels in test_loader:
    images = images.float().to(device)
    labels = labels.float().to(device)
    outputs = model(images, win)
    _, predicted = torch.max(outputs[:num_to_test,:].data, 1)
    _, reference = torch.max(labels[:num_to_test,:].data, 1)
    correct = (predicted == reference).sum()
    break

print('Test Accuracy of the model on the 10000 test images: %.3f' % (100 * correct.float() / num_to_test))

#### SpiNNaker: 50 muestras, 58%